# Model Strangelog
Statistical approach to label anomalies in event logs based on percentile analysis of tokenized events.

It returns a labelled dataframe from a test df_trace with new column 'strangeness' for each event:

| Strangeness | Meaning | comment |
|--------|---------|---------|
| 0      | Boring  | Observed in 80% or more of the traces | 
| 1      | Common  | Observed in 20% - 80% of the traces   | 
| 2      | Rare    | Observed in 20% or less of the traces | 
| 3      | Unexpected | Never seen in the subset | 

**Requires**
- log dataset from high level task
- LogColor instance pretrained with similar dataset
- A subset column with a partition of similar task executions. For example, TPL_ID.

### Example: traces grouped by TPL_ID

In [1]:
# Init the public dataset
from eliana.datasets import ParlogsObservations
logs = ParlogsObservations(system="GRAVITY", period="1w")

In [2]:
# Partition in success and errors
meta_success = logs.meta[ logs.meta['ERROR'] == False ]
meta_errors  = logs.meta[ logs.meta['ERROR'] == True ]
print(f'In {logs.system}, from {len(logs.meta)} executions, Success: {len(meta_success)}; Error: {len(meta_errors)}')

In GRAVITY, from 152 executions, Success: 128; Error: 24


In [3]:
# Train Log Colorizer
traces = logs.traces()
print(f'Loaded {len(traces)} traces')

Loaded 316207 traces


In [4]:
from eliana.preprocessing import LogColorizer, VltTokenizer
colorizer = LogColorizer(tokenizer=VltTokenizer(to_lowercase=False))
_ = colorizer.fit(traces, warn=True)

## Train Strangelog model

In [5]:
from eliana.models import Strangelog
subset_col='TPL_ID'

model = Strangelog(tokenizer=colorizer, subset_col=subset_col)
model.fit(meta_success, traces, warn=True)

## Predict for SUCCESS executions

SUCCESS should have very low instances of rare (2) and unexpected (3) events

| Strangeness | Meaning | comment |
|--------|---------|---------|
| 0      | Boring  | Observed in 80% or more of the traces | 
| 1      | Common  | Observed in 20% - 80% of the traces   | 
| 2      | Rare    | Observed in 20% or less of the traces | 
| 3      | Unexpected | Never seen in the subset | 

In [6]:
for row in meta_success.sample(5).itertuples():
    print(f"Strangeness for {row.TPL_ID} (Error = {row.ERROR})")
    labelled = model.predict(row.TPL_ID, traces[ traces['trace_id'] == row.Index ] )
    display( labelled.groupby('strangeness').size().reset_index(name='count'))


Strangeness for GRAVITY_gen_tec_RoofPos (Error = False)


,strangeness,count
0,0,112


Strangeness for GRAVITY_gen_tec_AcqFibA (Error = False)


,strangeness,count
0,0,696
1,1,5


Strangeness for GRAVITY_gen_cal_init (Error = False)


,strangeness,count
0,0,306
1,1,389
2,2,2


Strangeness for GRAVITY_gen_cal_init (Error = False)


,strangeness,count
0,0,1526
1,1,1013


Strangeness for GRAVITY_gen_cal_init (Error = False)


,strangeness,count
0,0,759
1,1,971
2,2,30


## Predict for ERROR executions

ERRORS should have hign number of rare (2) and unexpected (3) events. And in general, the content of the unexpected should notify about the cause of errors.

| Strangeness | Meaning | comment |
|--------|---------|---------|
| 0      | Boring  | Observed in 80% or more of the traces | 
| 1      | Common  | Observed in 20% - 80% of the traces   | 
| 2      | Rare    | Observed in 20% or less of the traces | 
| 3      | Unexpected | Never seen in the subset | 

In [7]:
for row in meta_errors.sample(5).itertuples():
    print(f"-------\nUnexpected events in {row.TPL_ID} (Error = {row.ERROR})")
    trace = traces[ traces['trace_id'] == row.Index ]
    labelled = model.predict(row.TPL_ID, trace )
    unexpected = labelled[ labelled['strangeness'] == 3 ]
    unexpected['color'] = unexpected['event'].apply(colorizer.tokenize)
    print()
    print( unexpected['color'].unique()[:20] )
    display( labelled.groupby('strangeness').size().reset_index(name='count'))


-------
Unexpected events in GRAVITY_single_acq (Error = True)

['FLOG logManager UNFORSEEN template error'
 'ERR bob_{} Error during setup states red']


,strangeness,count
0,0,1277
1,1,284
2,2,4
3,3,3


-------
Unexpected events in GRAVITY_single_acq (Error = True)

['FLOG logManager UNFORSEEN template error'
 'LOG bob_{} ERROR performing command SETUP INFO Preset telescopes to FREE position red'
 'LOG bob_{} INFO Stop Coude auto guiding red' 'ERR bob_{} PRESET red']


,strangeness,count
0,0,163
1,1,98
2,3,4


-------
Unexpected events in GRAVITY_dual_acq (Error = True)

['LOG bob_{} Configure PSD ATM to start PSF scan blue'
 'LOG bob_{} Beam 1x psf {} 011ttp {} {} difference {} green4'
 'LOG bob_{} Beam 1y psf {} 011ttp {} {} difference {} green4'
 'LOG bob_{} Beam 2x psf {} 011ttp {} {} difference {} green4'
 'LOG bob_{} Beam 2y psf {} 011ttp {} {} difference {} green4'
 'LOG bob_{} Beam 3x psf {} 011ttp {} {} difference {} green4'
 'LOG bob_{} Beam 3y psf {} 011ttp {} {} difference {} green4'
 'LOG bob_{} Beam 4x psf {} 011ttp {} {} difference {} green4'
 'LOG bob_{} Beam 4y psf {} 011ttp {} {} difference {} green4'
 'LOG bob_{} Send command SETPTFI {} REL TipTiltPiston {} {} to process gvttpControl@wgv'
 'LOG bob_{} PSF Scan was applied for all beams'
 'LOG bob_{} Pick Manually'
 'LOG bob_{} Send command CONVABS {} FiberPosB AcqCam {} {} to process gvctuControl@wgv'
 'LOG bob_{} Send command CONVABS {} AcqCam FiberPosB {} {} to process gvctuControl@wgv'
 'LOG bob_{} SETUP noExposure noEx

,strangeness,count
0,0,4613
1,2,217
2,3,79


-------
Unexpected events in GRAVITY_dual_acq (Error = True)

['FLOG logManager UNFORSEEN template error' 'ERR bob_{} PRESET red'
 'LOG bob_{} ERROR performing command SETUP INFO Preset telescopes to FREE position red'
 'LOG bob_{} INFO Stop Coude auto guiding red']


,strangeness,count
0,0,290
1,2,19
2,3,4


-------
Unexpected events in GRAVITY_single_acq (Error = True)

['ERR bob_{} seqERR_REPLY_TIMEOUT reply on cmd{} CALIBSN to lgvft2 gviopdcServer timed out after {} ms'
 'FEVT logManager TPL EXEC START Template execution started']


,strangeness,count
0,0,1643
1,1,460
2,2,16
3,3,2
